In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
# soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [20]:
url = f'https://www.yesstyle.com/es/mujer-limpiadores-faciales/list.html/bcc.14335_bpt.46#/bpt=46&sb=136&pn=1&l=17&s=10&bcc=14335&bt=37' 
r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml')


In [95]:
a = soup.find_all('a', href=True)

In [96]:
type(a)

list

In [108]:
url = []
for item in soup.find_all('a', href=True):
    url.append(item['href'])

In [109]:
url

['javascript:void(0)',
 'https://www.yesstyle.com/es/help/section.html/hsi.727##1521',
 'https://www.yesstyle.com/es/women.html',
 'https://www.yesstyle.com/es/women.html',
 'https://www.yesstyle.com/es/beauty.html',
 'https://www.yesstyle.com/es/mujer-belleza/list.html/bcc.14072_bpt.46?oc=1&sb=158',
 'https://www.yesstyle.com/es/mujer-belleza/list.html/bcc.14072_bpt.46?oc=2',
 'https://www.yesstyle.com/es/mujer-belleza/list.html/bcc.14072_bpt.46?oc=23',
 '/es/special-offers.html',
 '/es/japanese-beauty-cosmetics',
 '/es/taiwanese-beauty-cosmetics',
 '/es/us-beauty-cosmetics',
 'https://www.yesstyle.com/es/mujer-belleza/list.html/bcc.14072_bpt.46',
 'https://www.yesstyle.com/es/mujer-accesorios-y-complementos-de-belleza/list.html/bcc.14105_bpt.46',
 'https://www.yesstyle.com/es/mujer-cuidado-facial/list.html/bcc.14107_bpt.46',
 'https://www.yesstyle.com/es/mujer-cuidado-del-cuerpo/list.html/bcc.14106_bpt.46',
 'https://www.yesstyle.com/es/mujer-cuidado-solar/list.html/bcc.15396_bpt.46'

In [119]:
final=[]
for elements in url:
    if (re.match(r'(pid)', elements)):
        final.append(elements)
    

In [120]:
final

    

[]

In [106]:
type(links)

list

In [107]:
links

[]

In [10]:
cleansers_url = []
for i in range (1, 3):
    url = f'https://www.yesstyle.com/es/mujer-limpiadores-faciales/list.html/bcc.14335_bpt.46#/bpt=46&sb=136&pn={i}&l=17&s=10&bcc=14335&bt=37' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', data-ng-repeat='lotinfoitem in lotinfoitems')
    print(cleansers_list)
    for item in cleansers_list:
        for link in item.find_all('a', class_='ng-isolate-scope', href=True): 
            cleansers_url.append(link['href'])
print(len(cleansers_url))

[]
[]


In [8]:
print(cleansers_list)

[]


In [ ]:
for item in cleansers_list:
        for link in item.find_all('a', class_='ng-isolate-scope', href=True): 
            cleansers_url.append(link['href'])
print(len(cleansers_url))

In [91]:
for i in range (4, 20):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='product-item-info')
    for item in cleansers_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            cleansers_url.append(link['href'])

In [93]:
for i in range (20, 41):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='product-item-info')
    for item in cleansers_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            cleansers_url.append(link['href'])

In [95]:
for i in range (41, 87):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='product-item-info')
    for item in cleansers_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            cleansers_url.append(link['href'])

In [96]:
print(len(cleansers_url))

1032


In [99]:
with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [27]:
#test_url = 'https://www.stylevana.com/en_EU/laneige-lip-sleeping-mask.html'

LANEIGE - Lip Sleeping Mask


In [ ]:
cleansers = []

In [115]:
for link in cleansers_url[891:1032]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    name = soup.find('h1', class_='product-name-h1').text
    details = (soup.find('div', id='product-view-details').text.strip()) # details
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        ingred = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingred:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        ingredients = 'no info'
        
    cleansers_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients}
    
    cleansers.append(cleansers_info)

In [116]:
print(len(cleansers))

1032


## 3. PASARLO A DATAFRAME

In [117]:
cleansers_df = pd.DataFrame(cleansers)
cleansers_df

,name,details,ingredients
0,iUNIK - Calendula Complete Cleansing Oil - 200ml,Product Type: Cleansing Oil\n\n\nSkin Concern:...,"Centella Asiatica Leaf Water,Water,TEA-Cocoyl ..."
1,Krave - Matcha Hemp Hydrating Cleanser 120ml,Product Type: Cleansing Gel\n\n\nSkin Concern:...,"Centella Asiatica Leaf Water,Water,TEA-Cocoyl ..."
2,COSRX - Low pH Good Morning Gel Cleanser - 150ml,Cruelty Free: Yes\n\n\nProduct Type: Cleansing...,"Water, Cocamidopropyl Betaine, Sodium Lauroyl ..."
3,heimish - All Clean Balm,Cruelty Free: Yes\n\n\nProduct Type: Cleansing...,"Ethylhexyl Palmitate, Cetyl Ethylhexanoate, PE..."
4,SOME BY MI - AHA-BHA-PHA 30days Miracle Acne C...,Product Type: Cleansing Water\n\n\nSkin Concer...,"Ethylhexyl Palmitate, Cetyl Ethylhexanoate, PE..."
...,...,...,...
1027,Haruharu WONDER - Black Rice Moisture Deep Cle...,Product Type: Cleansing Oil\n\n\nSkin Type: Co...,"Water, Glycerin, Acer Saccharum (Sugar Maple) ..."
1028,iUNIK - Centella Bubble Cleansing Foam - 150ml,Product Type: Cleansing Foam\n\n\nSkin Concern...,"Centella Asiatica Leaf Water,Water,TEA-Cocoyl ..."
1029,heimish - All Clean Green Foam - 150ml,Cruelty Free: Yes\n\n\nProduct Type: Cleansing...,"Centella Asiatica Leaf Water,Water,TEA-Cocoyl ..."
1030,SOME BY MI - Bye Bye Blackhead 30days Miracle ...,Product Type: Cleansing Foam\n\n\nSkin Concern...,"Centella Asiatica Leaf Water,Water,TEA-Cocoyl ..."


In [118]:
cleansers = cleansers_df.to_csv('data/cleansers.csv')

{'name': 'LANEIGE - Lip Sleeping Mask', 'details': 'Product Type: Lip Mask\n\n\nSkin Type: Combination, Dry, Normal, Oily, Sensitive', 'ingredients': 'Diisostearyl Malate, Hydrogenated Polyisobutene, Phytosteryl/Isostearyl/Cetyl/Stearyl/Behenyl Dimer Dilinoleate, Hydrogenated Poly(C6-14 Olefin), Polybutene, Microcrystalline Wax, Shea Butter, Synthetic Wax, Candelilla Wax, Sucrose Tetrastearate Triacetate, Butylene/Ethylene/Styrene Copolymer, Ethylene/Propylene/Styrene Copolymer, Mica, Fragrance, Astrocaryum Murumuru Seed Butter, Dimethicone, Titanium Dioxide (CI 77891), Polyglyceryl-3 Diisostearate, Dehydroacetic Acid, Polyglyceryl-2 Triisostearate, Methicone, Carnauba Wax, Polyhydroxystearic Acid, Water, Potassium Alginate, Propanediol, Glycerin, Ethanol, Lycium Chinense Fruit Extract, Vaccinium Macrocarpon (Cranberry) Fruit Extract, Fragaria Ananassa (Strawberry) Fruit Extract, Rubus Idaeus (Raspberry) Fruit Extract, Sapindus Mukorossi Fruit Extract, Vaccinium Angustifolium (Blueberr